In [3]:
import numpy as np
import multiprocessing as mp
import os

In [4]:
def func1(x):
    x = np.array(x)
    i = np.array(range(1,len(x)+1))
    result = 1/40*np.sum(x**2) + 1 - np.prod(np.cos(x/i))
    
    return result

In [5]:
# Zmieniam znak przy 20 na koncu równania (z +20 na -20)
def func2(x):
    x = np.array(x)
    n = len(x)
    result_1 = 20 * np.exp(-0.2 * np.sqrt(1/n * np.sum(x**2)))
    result_2 = - np.exp(1/n * np.sum(np.cos(2*np.pi*x))) - 20 + np.e
    return result_1 + result_2

Wrzuciłem funkcje w wolframa:

https://www.wolframalpha.com/input/?i=20*e%28-0.2*sqrt%28x%5E2%29%29+-+exp%28cos%282*pi*x%29%29+%2B+20+%2B+e

Wygląda na to, że powinno też być max, a nie min ;)

In [6]:
def initial_pop_generation(x, n, start_wth_zeros=False, min_value_n=None,
                           max_value_n=None):
    '''
    x : int
        parzysta liczba osobników
    n : int
        parzysta liczba genów
    start_wth_zeros : bool
        True - początkowe cechy osobnika równe zero
        False - losowo dobrane cechy początkowe
    min_value_n : int
        dolne ogarniczenie wartości genu
    max_value_n : int
        górne ograniczenie wartości genu
    '''
    population = []
    if start_wth_zeros:
        for i in range(x):
            individual = np.array([0 for i in range(0, n)])
            population.append(individual)
    else:
        if (min_value_n == None or max_value_n == None):
            raise Exception('Set min&max value for n or set start_with_zeros to True')
        
        for i in range(x):
            individual = np.array([np.random.randint(min_value_n, max_value_n) for i in range(0, n)])
            population.append(individual)

    return population

In [7]:
def crossing(indiv1, indiv2,n):
    '''
    indiv1

    indiv2
    '''
    #losowanie locus podzialu
    locus = np.random.randint(0, n)
    embryo1 = np.concatenate((indiv1[:locus],indiv2[locus:]))
    embryo2 = np.concatenate((indiv2[:locus],indiv1[locus:]))

    return embryo1, embryo2

In [8]:
def mutate(individual, p_mutate):
    '''
    individuals : [int]
        lista genow
    p_mutate : float
        prawdopodobienstwo mutacji
    '''

    for i in range(len(individual)):
        r = np.random.rand()
        if r < p_mutate/2:
            individual[i] -= 1
        elif r < p_mutate:
            individual[i] += 1
        
    return individual
        

In [9]:
def optimize(population, x, n, function, min_or_max,end_optim, p_mutate=0.2):
    '''
    population : list of numpy arrays
        zbiór wszystkich osobników
    '''
    best_individual_scores = []
    iteration = 0
    iteration_dict = {}    
    score_dict = {}    
    while True:
        iteration += 1
        
        #krzyzowanie i mutacja
        pop_score_dict = {}
        for i in range(0, x, 2):
            crossing_result = crossing(population[i], population[i+1], n)
            for embryon in crossing_result:
                population.append(mutate(embryon,p_mutate))  
        
        scores = []
        for individual in population:
            score = function(individual)
            scores.append(score)
            pop_score_dict[tuple(individual)] = float(score)
        
        if min_or_max == 'max':
            reverse=True
        elif min_or_max == 'min':
            reverse=False
        else:
            raise Exception('min_or_max parameter invalid. Type "min" or "max".')
            
        population_score = sorted(zip(population, scores), key=lambda x:x[1], reverse=reverse)
        best_individual_gen = population_score[0][0]
        best_individual_score = population_score[0][1]
        
        best_individual_scores.append(best_individual_score)
            
        sorted_population, _ = zip(*population_score)
        population = list(sorted_population[:x])
        
        #print(f'Iteration: {iteration} | Best score: {best_individual_score}')
        
        iteration_dict[int(f'{iteration}')] = best_individual_score
        score_dict[int(f'{iteration}')] = pop_score_dict
        
        # zakończ jeśli nie ma poprawy w 100 ostatnich iteracjach (?)
        if len(set(best_individual_scores[-end_optim:])) == 1 and iteration > 10:
            break
        
        
    return best_individual_gen, best_individual_score, iteration_dict, score_dict
        

In [10]:
def optimize_mp(population, x, n, function, 
         min_or_max, end_optim, p_mutate,
                shared_list, result_list):
         
    best_individual_scores = []
    iteration = 0
    while True:
        iteration += 1
         
        if min_or_max == 'max':
            reverse=True
        elif min_or_max == 'min':
            reverse=False
        else:
            raise Exception('min_or_max parameter invalid. Type "min" or "max".')
        
        #Krzyżowanie i mutacja
        for i in range(0, x, 2):
            crossing_result = crossing(population[i], population[i+1], n)
            for embryon in crossing_result:
                population.append(mutate(embryon,p_mutate))
        
        #Ocena
        scores = []
        for individual in population:
            score = function(individual)
            scores.append(score)
        
        # Co 20 iteracji weź najlepszych osobników od innych procesów
        if shared_list and iteration%20==1:
            for i in range(len(shared_list)):
                if os.getpid() not in shared_list[i][0]:
                    population.append(shared_list[i][1])
                    scores.append(shared_list[i][2])
                    # Wrzucam id procesu, żeby już wiecej nie pobrać tego osobnika
                    shared_list[i][0].append(os.getpid())
        
        population_score = sorted(zip(population, scores), key=lambda x:x[1], reverse=reverse)        
        best_individual_gen = population_score[0][0]
        best_individual_score = population_score[0][1]
        best_individual_scores.append(best_individual_score)
        
        # Co 20 iteracji zapisz najlepszego osobnika na wspólną listę
        # Lista id procesów na początku, żeby nie duplikować osobników tej samej populacji
        if iteration % 20 == 0:
            if shared_list: # Jeżeli lista nie jest pusta, to dodaje osobnika tylko gdy jest lepszy
                best_shared_score = shared_list[len(shared_list)-1][2]
                if ( 
                     (min_or_max == 'max' and best_individual_score > best_shared_score) or 
                     (min_or_max == 'min' and best_individual_score < best_shared_score)
                   ):
                    shared_list.append([manager.list([os.getpid()]),best_individual_gen, 
                                best_individual_score])
            else:
                shared_list.append([manager.list([os.getpid()]),best_individual_gen, 
                                best_individual_score])
         
        sorted_population, _ = zip(*population_score)
        population = list(sorted_population[:x])
        
#         if iteration%50==0:
#             print(f'Iteration: {iteration} | Best score: {best_individual_score} | Process: {os.getpid()}')
        
        # zakończ jeśli nie ma poprawy w 100 ostatnich iteracjach (?)
        if len(set(best_individual_scores[-end_optim:])) == 1 and iteration > 10:
            result_list.append((best_individual_score, best_individual_gen))
            break
            

In [12]:
def run_mp_alg(manager, x,n,function,
               min_or_max,end_optim=100, p_mutate=0.2,p_num=mp.cpu_count(),
               start_wth_zeros=False, min_value_n=None,
                           max_value_n=None):
    '''
    manager : multiprocessing.Manager
        umożliwia przekazywanie wyników pomiędzy procesami
    x : int
        liczba osobników w populacji
    n : int
        liczba genów każdego osobnika
    function : function
        funkcja oceny osobnika
    min_or_max : String
        'min' jeżeli wartość funkcji oceny ma być zminimalizowana
        'max' jeżeli maksymalizowana
    p_mutate : float
        pradopodobieństwo mutacji; dommyślnie równe 0.2
    p_num : int
        liczba procesów; domyślnie równa liczbie CPU
    start_wth_zeros : bool
        True - początkowe cechy osobnika równe zero
        False - losowo dobrane cechy początkowe
    min_value_n : int
        dolne ogarniczenie wartości genu
    max_value_n : int
        górne ograniczenie wartości genu
    end_optim  : int
        ilość iteracji po których brak poprawy najlepszego osobnika 
        zakończy działanie algorytmu
    '''
    shared_list = manager.list()
    result_list = manager.list()
    
    populations = [initial_pop_generation(x,n,start_wth_zeros,min_value_n,max_value_n) 
                   for i in range(p_num)]
    processes = [mp.Process(target=optimize_mp, 
                                  args=[populations[i], x, n, function, 
                                  min_or_max,end_optim, p_mutate,
                                  shared_list,result_list]) for i in range(p_num)]
        
    for process in processes:
        process.start()

    for process in processes:
        process.join()
    
    if min_or_max == 'max':
        result = sorted(result_list, key=lambda x:x[0], reverse=True)[0] 
    else:
        result = sorted(result_list, key=lambda x:x[0], reverse=False)[0]
        
    return result

In [9]:
def optimize_pymp(population, x, n, function, min_or_max,end_optim, shared_arr, p, p_mutate=0.2):
    '''
    population : list of numpy arrays
        zbiór wszystkich osobników
    '''
    best_individual_scores = []
    iteration = 0
    iteration_dict = {}    
    score_dict = {}    
    while True:
        iteration += 1
        
        #krzyzowanie i mutacja
        pop_score_dict = {}
        for i in range(0, x, 2):
            crossing_result = crossing(population[i], population[i+1], n)
            for embryon in crossing_result:
                population.append(mutate(embryon,p_mutate))  
        
        scores = []
        for individual in population:
            score = function(individual)
            scores.append(score)
            pop_score_dict[tuple(individual)] = float(score)
        
        if min_or_max == 'max':
            reverse=True
        elif min_or_max == 'min':
            reverse=False
        else:
            raise Exception('min_or_max parameter invalid. Type "min" or "max".')
            
        population_score = sorted(zip(population, scores), key=lambda x:x[1], reverse=reverse)
        best_individual_gen = population_score[0][0]
        best_individual_score = population_score[0][1]
        
        best_individual_scores.append(best_individual_score)
            
        sorted_population, _ = zip(*population_score)
        population = list(sorted_population[:x])
        
#         np.append(shared_arr,best_individual_score)
#         shared_arr[-1] = best_individual_score
#         print(best_individual_score)
#         print(shared_arr)
        #print(f'Iteration: {iteration} | Best score: {best_individual_score}')
        
        iteration_dict[int(f'{iteration}')] = best_individual_score
        score_dict[int(f'{iteration}')] = pop_score_dict
        
        # zakończ jeśli nie ma poprawy w 100 ostatnich iteracjach (?)
        if len(set(best_individual_scores[-end_optim:])) == 1 and iteration > 10:
            break
        
        
    return best_individual_gen, best_individual_score, iteration_dict, score_dict
        

In [21]:
def optimize_parallel(population, x, n, function, 
                      min_or_max,end_optim, 
                      communication_queue, 
                      in_queue, out_queue,
                      result_queue,
                      p_mutate=0.2):
    
    best_individual_scores = []
    iteration = 0
    while True:
        iteration += 1
        #krzyzowanie i mutacja
        embryons = []
        for i in range(0, x, 2):
            crossing_result = crossing(population[i], population[i+1], n)
            for embryon in crossing_result:
                population.append(mutate(embryon,p_mutate))
                
        scores = []
        for individual in population:
            score = function(individual)
            scores.append(score)

        if min_or_max == 'max':
            reverse=True
        elif min_or_max == 'min':
            reverse=False
        else:
            raise Exception('min_or_max parameter invalid. Type "min" or "max".')
        
        
        print(in_queue, out_queue,'\n')
        if iteration%50==0:
            in_queue.put((best_individual_gen,best_individual_score))
            print('Putting',str(best_individual_score),'on',in_queue)
    
        if not out_queue.empty():
            print('Queue is not empty')
            data = out_queue.get(False)
            print('Got', str(data))
            
        population_score = sorted(zip(population, scores), key=lambda x:x[1], reverse=reverse)
        best_individual_gen = population_score[0][0]
        best_individual_score = population_score[0][1]
        
        best_individual_scores.append(best_individual_score)
        sorted_population, _ = zip(*population_score)
        population = list(sorted_population[:x])

        if iteration%10==0:
            print(f'Iteration: {iteration} | Best score: {best_individual_score}')
            
        # zakończ jeśli nie ma poprawy w n ostatnich iteracjach
        if len(set(best_individual_scores[-end_optim:])) == 1 and iteration > 10:
            break
        
    result_queue.put((best_individual_gen, best_individual_score))
        
    return best_individual_gen, best_individual_score